In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

In [11]:
url="https://www.imdb.com/list/ls574185453/?sort=list_order,asc&st_dt=&mode=detail&page=1&ref_=ttls_vm_dtl"

In [12]:
response = requests.get(url)
response

<Response [200]>

#### Parse the HTML content with BeautifulSoup

In [13]:
soup = BeautifulSoup(response.content,'html.parser')

In [14]:
type(soup)

bs4.BeautifulSoup

In [15]:
movies = soup.find_all('div' ,class_='lister-item-content')
len(movies)

100

In [16]:
movies[0]

<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6710474/">Everything Everywhere All at Once</a>
<span class="lister-item-year text-muted unbold">(2022)</span>
</h3>
<p class="text-muted text-small">
<span class="certificate">A</span>
<span class="ghost">|</span>
<span class="runtime">139 min</span>
<span class="ghost">|</span>
<span class="genre">
Action, Adventure, Comedy            </span>
</p>
<div class="ipl-rating-widget">
<div class="ipl-rating-star small">
<span class="ipl-rating-star__star">
<svg class="ipl-icon ipl-star-icon" fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h24v24H0z" fill="none"></path>
<path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path>
<path d="M0 0h24v24H0z" fill="none"></path>
</svg>
</span>
<span class="ipl-rating-star__rating">7.8</span>
</div>
<

In [103]:
final =pd.DataFrame()
for i in range(1,11):
    url="https://www.imdb.com/list/ls574185453/?sort=list_order,asc&st_dt=&mode=detail&page={}&ref_=ttls_vm_dtl".format(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    movies = soup.find_all('div' ,class_='lister-item-content')
    
    movie_name=[]
    running_time=[]
    metascore=[]
    release_year=[]
    imdb_rating=[]
    description=[]
    directors=[]
    stars=[]
    votes=[]
    gross=[]
    genre=[]
    
    for movie in movies:
# fetching movies name
        movie_name.append(movie.find('a').text.strip())
        
        
# fetching running time
        try:
            running_time.append(movie.find('span',class_='runtime').text.strip())
        except:
            running_time.append(np.nan)

            
# fetching metascore
        try:
            metascore.append(movie.find('span',class_='metascore').text.strip())

        except:
            metascore.append(np.nan)
            
            
# fetching genre        
        try:
            genre.append(movie.find('span',class_='genre').text.strip())
        except:
            genre.append(np.nan)

            
# fetching release year
        try:
            year=movie.find('span',class_='lister-item-year text-muted unbold')
            pattern = r'\D'
            modified_year=[re.sub(pattern, '', item) for item in year]
            release_year.append(modified_year[0])
        except:
            release_year.append(np.nan)

            
# fetching imdb-rating            
        imdb_rating.append(movie.find('span',class_='ipl-rating-star__rating').text.strip())   

    
# fetching description
        description.append(movie.find('p',class_='').text.strip())

    
# fetching votes    
        try:
            votes.append(movie.find_all('span', attrs={'name':'nv','data-value':True})[0].text.strip())
        except:
            votes.append(np.nan)

    
# fetching gross 
        try:
            gross.append(movie.find_all('span', attrs={'name':'nv','data-value':True})[1].text.strip())
        except:
            gross.append(np.nan)



        movies_dict={'Movie_Name':movie_name,
                     'Genre':genre,
                    'Running_Time':running_time,
                    'Metascore':metascore,
                    'Release_year':release_year,
                    'IMDb_Rating':imdb_rating,
                    'Movie_Description':description,
                     'Votes':votes,
                     'Gross':gross
                        }
        df = pd.DataFrame(movies_dict)
        
        
    final=pd.concat([final,df],ignore_index=True)

In [104]:
final

,Movie_Name,Genre,Running_Time,Metascore,Release_year,IMDb_Rating,Movie_Description,Votes,Gross
0,Everything Everywhere All at Once,"Action, Adventure, Comedy",139 min,81,2022,7.8,A middle-aged Chinese immigrant is swept up in...,"470,769",$72.86M
1,Spider-Man: Into the Spider-Verse,"Animation, Action, Adventure",117 min,87,2018,8.4,Teen Miles Morales becomes the Spider-Man of h...,"619,109",$190.24M
2,Shin seiki Evangelion Gekijô-ban: Air/Magokoro...,"Animation, Action, Drama",87 min,NaN,1997,8.1,Concurrent theatrical ending of the TV series ...,"59,869",NaN
3,Star Wars: Episode V - The Empire Strikes Back,"Action, Adventure, Fantasy",124 min,82,1980,8.7,After the Rebels are overpowered by the Empire...,"1,335,275",$290.48M
4,Stalker,"Drama, Sci-Fi",162 min,85,1979,8.1,A guide leads two men through an area known as...,"139,753",$0.23M
...,...,...,...,...,...,...,...,...,...
995,Fiend Without a Face,"Horror, Sci-Fi",77 min,NaN,1958,6.1,A scientist's thoughts materialize as an army ...,"4,609",NaN
996,Battlestar Galactica: Blood & Chrome,"Action, Sci-Fi",91 min,NaN,2012,6.9,The adventures of young William Adama in the F...,"18,255",NaN
997,Alien Nation,"Action, Sci-Fi",91 min,45,1988,6.3,"In 1988, Earth makes the first contact with an...","16,265",$25.22M
998,Spontaneous Combustion,"Horror, Sci-Fi, Thriller",97 min,NaN,1989,4.8,A young man finds out that his parents had bee...,"2,856",NaN


In [ ]:
release_year=[]
for movie in movies:
    year=movie.find('span',class_='lister-item-year text-muted unbold')
    pattern = r'\D'
    modified_year=[re.sub(pattern, '', item) for item in year]
    release_year.append(modified_year[0])
    
        
release_year        

In [ ]:
votes=[]
gross=[]

for movie in movies:
    try:
        votes.append(movie.find_all('span', attrs={'name':'nv','data-value':True})[0].text.strip())
    except:
        votes.append(np.nan)
        
    try:
        gross.append(movie.find_all('span', attrs={'name':'nv','data-value':True})[1].text.strip())
    except:
        gross.append(np.nan)

votes

In [ ]:
directors=[]

from lxml import html

# html.fromstring(response.content) creates an ElementTree object (tree) from the HTML content. 
# This object allows you to apply XPath expressions to navigate through and extract data from the HTML structure.
for movie in movies:
#     print(movie)
# movies is a list of div tags, so we need to extract the html content from it 
    tree = html.fromstring(str(movie))
    directors=tree.xpath('//*[@id="main"]/div/div[3]/div[3]/div[1]/div[2]/p[3]/a[1]/text()')

directors     

In [9]:
from lxml import etree as et
director=[]


for movie in movies:
    tree = et.ElementTree(movie)
    director.append(tree.xpath('//*[@id="main"]/div/div[3]/div[3]/div[1]/div[2]/p[3]/a/text()'))
director

TypeError: Argument 'element' has incorrect type (expected lxml.etree._Element, got Tag)

In [107]:
filename = 'top_IMDb_Sci-Fi_movies.csv'
final.to_csv(filename,index=False)

In [19]:
directors=[]
from lxml import html

for movie in movies:
    tree = html.fromstring(str(movie))
    tree.xpath('/html/body/div[1]/div/div[2]/div/div[1]/div/div[3]/div[3]/div[1]/div[2]/p[3]/a')
    

In [20]:
tree = html.fromstring(str(movies[0]))

In [21]:
tree.xpath('/html/body/div[1]/div/div[2]/div/div[1]/div/div[3]/div[3]/div[1]/div[2]/p[3]/a')

[]

In [23]:
import requests
from lxml import html

url = "https://www.imdb.com/list/ls574185453/?sort=list_order,asc&st_dt=&mode=detail&page=1&ref_=ttls_vm_dtl"

response = requests.get(url)
tree = html.fromstring(response.content)

movie_items = tree.xpath('//div[@class="lister-item-content"]')


for item in movie_items:
    title = item.xpath('.//h3[@class="lister-item-header"]/a/text()')[0].strip()
    directors = item.xpath('.//p[contains(text(), "Director")]/a/text()')
    actors = item.xpath('.//p[contains(text(), "Stars")]/a/text()')
    
    print("Movie:", title)
    print("Directors:", ", ".join(directors) if directors else "N/A")
    print("Actors:", ", ".join(actors) if actors else "N/A")
    print("-" * 40)


Movie: Everything Everywhere All at Once
Directors: Daniel Kwan, Daniel Scheinert, Michelle Yeoh, Stephanie Hsu, Jamie Lee Curtis, Ke Huy Quan
Actors: N/A
----------------------------------------
Movie: Spider-Man: Into the Spider-Verse
Directors: Bob Persichetti, Peter Ramsey, Rodney Rothman, Shameik Moore, Jake Johnson, Hailee Steinfeld, Mahershala Ali
Actors: N/A
----------------------------------------
Movie: Shin seiki Evangelion Gekijô-ban: Air/Magokoro wo, kimi ni
Directors: Hideaki Anno, Kazuya Tsurumaki, Megumi Ogata, Megumi Hayashibara, Yûko Miyamura, Kotono Mitsuishi
Actors: N/A
----------------------------------------
Movie: Star Wars: Episode V - The Empire Strikes Back
Directors: Irvin Kershner, Mark Hamill, Harrison Ford, Carrie Fisher, Billy Dee Williams
Actors: N/A
----------------------------------------
Movie: Stalker
Directors: Andrei Tarkovsky, Alisa Freyndlikh, Aleksandr Kaydanovskiy, Anatoliy Solonitsyn, Nikolay Grinko
Actors: N/A
--------------------------------

In [25]:
import requests
from lxml import html

url = "https://www.imdb.com/list/ls574185453/?sort=list_order,asc&st_dt=&mode=detail&page=1&ref_=ttls_vm_dtl"

response = requests.get(url)
tree = html.fromstring(response.content)

movie_items = tree.xpath('//div[@class="lister-item-content"]')

for item in movie_items:
    title = item.xpath('.//h3[@class="lister-item-header"]/a/text()')[0].strip()
    directors = item.xpath('.//p[contains(text(), "Director")]/a/text()')
    actors = item.xpath('.//p[contains(text(), "Stars")]/a/text()')
    
    print("Movie:", title)
    print("Directors:", ", ".join(directors) if directors else "N/A")
    print("Actors:", ", ".join(actors) if actors else "N/A")
    print("-" * 40)


Movie: Everything Everywhere All at Once
Directors: Daniel Kwan, Daniel Scheinert, Michelle Yeoh, Stephanie Hsu, Jamie Lee Curtis, Ke Huy Quan
Actors: N/A
----------------------------------------
Movie: Spider-Man: Into the Spider-Verse
Directors: Bob Persichetti, Peter Ramsey, Rodney Rothman, Shameik Moore, Jake Johnson, Hailee Steinfeld, Mahershala Ali
Actors: N/A
----------------------------------------
Movie: Shin seiki Evangelion Gekijô-ban: Air/Magokoro wo, kimi ni
Directors: Hideaki Anno, Kazuya Tsurumaki, Megumi Ogata, Megumi Hayashibara, Yûko Miyamura, Kotono Mitsuishi
Actors: N/A
----------------------------------------
Movie: Star Wars: Episode V - The Empire Strikes Back
Directors: Irvin Kershner, Mark Hamill, Harrison Ford, Carrie Fisher, Billy Dee Williams
Actors: N/A
----------------------------------------
Movie: Stalker
Directors: Andrei Tarkovsky, Alisa Freyndlikh, Aleksandr Kaydanovskiy, Anatoliy Solonitsyn, Nikolay Grinko
Actors: N/A
--------------------------------

In [26]:
import requests

api_key = "YOUR_OMDB_API_KEY"
url = f"http://www.omdbapi.com/?i=tt3896198&apikey={api_key}"  # Example URL, replace with the correct IMDb ID

response = requests.get(url)
data = response.json()

if data.get("Response") == "True":
    movie_title = data["Title"]
    directors = data["Director"].split(", ")
    actors = data["Actors"].split(", ")
    
    print("Movie:", movie_title)
    print("Directors:", ", ".join(directors))
    print("Actors:", ", ".join(actors))
else:
    print("Movie data not found.")


Movie data not found.
